# Image Masking

### Apply Masking to .dcm CT images to convert non-lung tissue to Houndsfield unit values representing air and save the processed images to disk

In [ ]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
import tensorflow as tf 
import matplotlib.pyplot as plt 
import random
from numpy import loadtxt
from scipy import stats

#Tensorflow packages
from tensorflow.keras.layers import (
    Dense, Dropout, Activation, Flatten, Input, BatchNormalization, GlobalAveragePooling2D, Add, Conv2D, MaxPooling2D, AveragePooling2D, 
    LeakyReLU, Concatenate) 

from tensorflow.keras import Model
from tensorflow.keras.utils import Sequence
import tensorflow.keras.applications as tfa
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import concatenate

#Sklearn packages
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

# Image processing packages
%matplotlib inline
import cv2
import pydicom
import pylibjpeg
import matplotlib.image as mpimg
from PIL import Image
from tabulate import tabulate
from IPython.display import display_html
import PIL.Image
#import gdcm
import gc
import missingno as msno 
from scipy.stats import pearsonr
from skimage.transform import resize
import re
import pylibjpeg

# Segmentation
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True)

def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
#Function to process images
def get_img(path):
    """This function receives a path as an input,
    and outputs a resized CT scan image that was
    also transformed into Hounsfield units. The information 
    needed to transform the voxels is included in the
    metadata contained in each CT scan"""

    pd = pydicom.dcmread(path)
    r = cv2.resize((pd.pixel_array - pd.RescaleIntercept) / (pd.RescaleSlope * 1000), (512, 512))
    mask = make_lungmask(r, display=False)
    return mask

In [ ]:
def make_lungmask(img, display = False):
  row_size = img.shape[0]
  col_size = img.shape[1]

  #Normalize
  mean = np.mean(img)
  std = np.std(img)
  img = img-mean
  img = img/std

  #Find the average pixel value near the lungs
  #to renormalize washed out images

  middle = img[int(col_size/5): int(col_size/5*4), int(row_size/5):int(row_size/5*4)]
  mean = np.mean(middle)
  max = np.max(img)
  min = np.min(img)

  #To improve threshold finding, move the underflow and overflow on the pixel spectrum
  img[img == max] = mean
  img[img == min] = mean

  #We use k-means to separate foreground (soft tissue/ bone) and banckground (lung/air)
  kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
  centers = sorted(kmeans.cluster_centers_.flatten())
  threshold = np.mean(centers)
  thresh_img = np.where(img<threshold, 1.0,0.0) #threshold the image

  #Erodde away
  eroded = morphology.erosion(thresh_img, np.ones([3,3]))
  dilation = morphology.dilation(eroded, np.ones([8,8]))
  labels = measure.label(dilation) #Different labels are displayed in different colors
  label_vals = np.unique(labels)
  regions = measure.regionprops(labels)
  good_labels = []
  for prop in regions:
    B = prop.bbox
    if B[2] - B[0] < row_size / 10*9 and B[3] - B[1] < col_size/10*9 and B[0] > row_size/5 and B[2] < col_size/ 5*4:
        good_labels.append(prop.label)
  mask = np.ndarray([row_size, col_size], dtype = np.int8)
  mask[:] = 0

  
   #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    
  for N in good_labels:
      mask = mask + np.where(labels==N,1,0)
  mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation
  if (display):
        fig, ax = plt.subplots(3, 2, figsize=[12, 12])
        ax[0, 0].set_title("Original")
        ax[0, 0].imshow(img, cmap='gray')
        ax[0, 0].axis('off')
        ax[0, 1].set_title("Threshold")
        ax[0, 1].imshow(thresh_img, cmap='gray')
        ax[0, 1].axis('off')
        ax[1, 0].set_title("After Erosion and Dilation")
        ax[1, 0].imshow(dilation, cmap='gray')
        ax[1, 0].axis('off')
        ax[1, 1].set_title("Color Labels")
        ax[1, 1].imshow(labels)
        ax[1, 1].axis('off')
        ax[2, 0].set_title("Final Mask")
        ax[2, 0].imshow(mask, cmap='gray')
        ax[2, 0].axis('off')
        ax[2, 1].set_title("Apply Mask on Original")
        ax[2, 1].imshow(mask*img, cmap='gray')
        #ax[2, 1].axis('off')
      
        plt.show()
  return mask*img    




In [ ]:
train = pd.read_csv('./train.csv')

In [ ]:
train_ID = train.Patient.unique()

In [ ]:
df_data={}
for p in train_ID:
    df_data[p] = os.listdir(f'./train/{p}/')
    print(p)
    for item in df_data[p]:
        try:
            img = get_img(f"./train/{p}/{item}")
            npn = item[:-4]+'.npy'
            np.save(f'./train_masked/{p}/{npn}', img)
        except:
            print(p,item)